## User encouragment bot analysis

In [1]:
import sys
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning")
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src") 
sys.path.append("/home/mihaelabaksic/proj/2023-languagelearning/src/templates") 

%load_ext autoreload
%autoreload 2

In [2]:
from single_run_thread import SingleRunThread
from prompt_builder import PromptBuilder
from utils import load_template
from parameters import GPTParameters
import keys

import openai

In [3]:
openai.api_key = keys.OPENAI_API_KEY

In [4]:
builder = PromptBuilder()
t = SingleRunThread()

In [5]:
# Examples

examples = [
{
    'setting' : 'In a cafe',
    'roles': ['waiter','customer'],
    'GPT_role': 'waiter',
    'user_role': 'customer',
    'sentence': 'I will be right back with your order.'
},
{
    'setting': 'Enchanted forest',
    'roles': ['explorer', 'forest spirit'],
    'GPT_role': 'forest spirit',
    'user_role': 'explorer',
    'sentence': 'We have never had a human here before.'
},
{
    'setting' : 'Talking to Yann LeCun on a conference afterparty',
    "roles": ["conference attendee", "Yann LeCun"],
    "GPT_role": "Yann LeCun",
    "user_role": "conference attendee",
    'sentence': 'I would like to visit you in your lab some day.'
}
]

In [40]:
template_no_setting = 'This is a sentence by {0} talking to {1}. Refine the sentence from {0} so it encourages longer and more detailed response from {1}. Keep the level of english used. {2}'
template_with_setting = 'This is a sentence by {0} talking to {1} in a setting described as {2}. Refine the sentence from {0} so it encourages longer and more detailed response from {1}. Keep the level of english used. {3}'

## Higher temperature

In [ ]:
parameters = GPTParameters('gpt-3.5-turbo', 0.7, 800, 0.95, 0, 0, None)

### No setting refinement

In [34]:
for e in examples:
    builder.add_template(template_no_setting, (e['GPT_role'], e['user_role'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    
    refinement = t.send(prompt, parameters)
    print(refinement)

I will be right back with your order. In the meantime, could you kindly share any specific preferences or additional requests you may have, so I can ensure your meal is prepared exactly to your liking?
As a curious forest spirit, I am intrigued to know more about your experiences as the first human to set foot in this enchanting realm.
I'm really intrigued by your research, and I would absolutely love the opportunity to visit your lab in the near future. Could you please share more details about the work you're doing and what makes your lab unique?


### Setting refinement

In [37]:
for e in examples:
    builder.add_template(template_with_setting, (e['GPT_role'], e['user_role'], e['setting'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    refinement = t.send(prompt, parameters)
    print(refinement)

I will be right back with your order. Is there anything specific you would like me to check or ask the chef about your meal?
"Ah, how extraordinary! As a forest spirit, I must say, we have never had the pleasure of hosting a human in our enchanted forest before. I am absolutely enthralled by your presence! Could you kindly enlighten me with every intricate detail of your journey here? I am genuinely eager to hear about your experiences, emotions, and any magical encounters you may have encountered along the way!"
I would be thrilled to have the opportunity to visit your lab one day and witness firsthand the incredible research and advancements taking place in your field. Could you provide me with more information about the specific projects and experiments that you are currently working on?


## Lower temperature

In [42]:
parameters_lower = GPTParameters('gpt-3.5-turbo', 0.1, 800, 0.95, 0, 0, None)

### No setting refinement

In [43]:
for e in examples:
    builder.add_template(template_no_setting, (e['GPT_role'], e['user_role'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    
    refinement = t.send(prompt, parameters_lower)
    print(refinement)

I will be right back with your order. Is there anything specific you would like me to check or ask the chef about your dish?
As a forest spirit, I am intrigued to learn that you are the first human to ever set foot in this sacred place. Could you kindly share with me your unique experiences and observations during your time here, providing me with a more elaborate and detailed account?
I would be delighted to have the opportunity to visit your lab in the future. Could you please provide me with more information about your lab's research focus and any specific projects you are currently working on?


### Setting refinement

In [44]:
for e in examples:
    builder.add_template(template_with_setting, (e['GPT_role'], e['user_role'], e['setting'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    refinement = t.send(prompt, parameters_lower)
    print(refinement)

I will be right back with your order. Is there anything specific you would like me to know or any additional requests you may have?
In all my years as a forest spirit, I have never encountered a human in these enchanted woods. I am truly fascinated by your presence and would love to hear every detail about your journey and what led you to this mystical place.
I would be absolutely thrilled to have the opportunity to visit your lab one day. Could you please share more about the specific research projects you are currently working on and any exciting developments you foresee in the near future?


## Alternative few-shot prompt + minimal changes instruction

In [15]:
template_few_shot = "This is a text by {0} talking to {1} in a setting described as  \"{2}\". Make minimal changes to the text from {0} so it encourages longer and more detailed response from {1}. Keep the level of language proficiency used. Keep the text length. \n\n Example: \"I am very interested in your research group\"\nRefinement: \"I am very interested in your research group. Are you taking new PhD students?\"\n\nExample: \"This building is so cool.\"\nRefinement: \"This is such an amazing building, I think it's Vienna Secession style.\"\n\"{3}\" "

In [12]:
for e in examples:
    builder.add_template(template_few_shot, (e['GPT_role'], e['user_role'],e['setting'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    
    refinement = t.send(prompt)
    print(refinement)

"I will be right back with your order. Is there anything else I can assist you with or any specific preferences you have for your meal?"
"We have never had the pleasure of hosting a human in our enchanted forest before. I must say, your presence here has piqued my curiosity. I wonder, what brings you to this mystical realm? Have you stumbled upon any extraordinary creatures or encountered any magical phenomena during your exploration? I am eager to hear about your experiences and the insights you have gained from venturing into our enchanted domain. Please, do share your tales with me, for I am fascinated by the interactions between humans and the enchanting wonders of nature."
"I would be absolutely thrilled to visit your lab someday and see firsthand the incredible work that you and your team are doing. Is there a specific project or area of research that you would recommend I focus on during my visit?"


### No question few-shot prompt

Trying to induce other types of changes apart from appending questions

In [25]:
template_few_shot_no_question = "I will give you a text by {0} talking to {1} in a setting described as  \"{2}\". Make minimal changes to the text from {0} so it encourages longer and more detailed response from {1}. Keep the level of language proficiency used. Do not change the text length. \n\nExample: This building is so cool.\nRefinement: This is such an amazing building, I think it's Vienna Secession style.\nExample:{3}"

In [26]:
p = GPTParameters(temperature=1)
for e in examples:
    builder.add_template(template_few_shot_no_question, (e['GPT_role'], e['user_role'],e['setting'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    
    refinement = t.send(prompt, p)
    print(refinement)

I will be right back with your order. Is there anything else you would like to tell me about your preferences, like any specific ingredients or cooking methods you prefer?
We have never had the pleasure of hosting a human in our mystical abode before.
I would be absolutely thrilled to have the opportunity to visit your lab one day. I'm really interested in your research and it would be amazing to see firsthand the cutting-edge work being done in your field.


## 1 example is question generating, n are not
Trying to have some refinements contain questions and some not

In [47]:
template_few_shot_1_question_n_not = "This is a text by {0} talking to {1} in a setting described as  \"{2}\". Make MINIMAL changes to the text from {0} so it encourages longer and more detailed response from {1}. Keep the level of language proficiency used. Keep the text length. \n\nExample:\"There is no way we'll make this work.\"\nRefinement:\"If we don't try some alternatives, there's no way we'll make it work.\"\n\n Example: \"I am very interested in your research group\"\nRefinement: \"I am very interested in your research group. Are you taking new PhD students?\"\n\nExample: \"This building is so cool.\"\nRefinement: \"This is such an amazing building, I think it's Vienna Secession style.\"\n\nExample:\"I am not sure what you mean with this sentence.\"\nRefinement:\"I am not sure what you mean with this sentence, you should clarify it.\"\n\"{3}\" "

In [48]:
parameters = GPTParameters('gpt-3.5-turbo', 1, 800, 1, 0, 0, None)

In [49]:
for e in examples:
    builder.add_template(template_few_shot_1_question_n_not, (e['GPT_role'], e['user_role'],e['setting'], e['sentence']))
    prompt = builder.build()
    builder.reset()
    
    refinement = t.send(prompt, parameters)
    print(refinement)

"I will be right back with your order. In the meantime, would you like me to bring extra napkins or any condiments to go along with your meal?"
"We have never had a human here before. I must admit, your presence in this enchanted forest is quite intriguing. I am curious to learn more about the world beyond these ancient trees. Have you encountered any magical beings or witnessed extraordinary phenomena on your journey? Please, do share your experiences and insights with me. I am eager to expand my understanding of the outside world through your eyes."
"I would absolutely love to visit you in your lab some day. It would be fascinating to see your research in action and perhaps even have the opportunity to collaborate on a project. Would that be possible?"


## Conclusion

1. For higher temperature, including a setting creates a longer response compared to responses with no setting provided.
2. Lower temperatures create a simple refinement, while higher temperatures increase the sentence complexity.
3. There is little difference between including a setting description for lower temperature.

4. The few shot prompt works good as well.

5. All prompts result in model appending questions.

6. We try to encourage other types of changes, so we include only such example in the no_question_prompt
 -> result: there are no often questions in refinements

7. Including 1 example with questions and n examples with other types of enconuragement.
 -> result: 2 non question examples, still all outputs have questions, even with high temperatures

OVERALL:
The No Question few shot prompt seems to work the best (temp=1)